### Initialize ellipsoids with DPD

In [ ]:
import gsd, gsd.hoomd 
import hoomd
import mbuild as mb
import numpy as np
import scipy.stats
import unyt as u
import warnings
from flowermd.base import Pack,Lattice, Simulation,BaseHOOMDForcefield,Polymer
from flowermd.base.system import System
from flowermd.library import EllipsoidForcefield, EllipsoidChain
from flowermd.utils import get_target_box_number_density
from flowermd.utils.constraints import create_rigid_ellipsoid_chain
from scipy.spatial.distance import pdist
warnings.filterwarnings('ignore')

In [ ]:
class EllipsoidChainRand(Polymer):
    def __init__(
        self,
        lengths,
        num_mols,
        lpar,
        bead_mass,
        bond_L=0.1,
        name="ellipsoid_chain",
    ):
        self.bead_mass = bead_mass
        self.lpar = lpar
        self.bond_L = bond_L
        self.bead_constituents_types = ["X", "A", "T", "T"]
        super(EllipsoidChainRand, self).__init__(
            lengths=lengths, num_mols=num_mols, name=name
        )

    def _build(self, length):
        bead = mb.Compound(name="ellipsoid")
        center = mb.Compound(pos=(0, 0, 0), name="X", mass=self.bead_mass / 4)
        head = mb.Compound(
            pos=(0, 0, self.lpar + (self.bond_L / 2)),
            name="A",
            mass=self.bead_mass / 4,
        )
        tether_head = mb.Compound(
            pos=(0, 0, self.lpar), name="T", mass=self.bead_mass / 4
        )
        tether_tail = mb.Compound(
            pos=(0, 0, -self.lpar), name="T", mass=self.bead_mass / 4
        )
        bead.add([center, head, tether_head, tether_tail])
        bead.add_bond([center, head])

        chain = mb.Compound()
        last_bead = None
        buffer = self.lpar + (self.bond_L / 2)
        density = 0.5
        N=50
        L = np.cbrt(N / density)
        for i in range(length):
            translate_by = np.random.uniform(low=-((L/2)-buffer), high=(L/2)-buffer, size=(3,))#adding random bead shift
            print(translate_by)
            this_bead = mb.clone(bead)
            this_bead.translate(by=translate_by)
            chain.add(this_bead)
            if last_bead:
                chain.add_bond([this_bead.children[0], last_bead.children[1]])
                chain.add_bond([this_bead.children[3], last_bead.children[2]])
            last_bead = this_bead
        chain.name = f"{self.name}_{length}mer"

        return chain

class DPD_FF(BaseHOOMDForcefield):
    def __init__(
        self,
        epsilon,
        lpar,
        lperp,
        A,
        gamma,
        kT,
        r_cut,
        angle_k=None,
        angle_theta0=None,
        bond_k=100,
        bond_r0=0.1,
    ):
        self.epsilon = epsilon
        self.lpar=lpar
        self.lperp=lperp
        self.gamma = gamma
        self.A = A
        self.kT = kT
        self.r_cut = r_cut
        self.angle_k = angle_k
        self.angle_theta0 = angle_theta0
        self.bond_k = bond_k
        self.bond_r0 = bond_r0
        hoomd_forces = self._create_forcefield()
        super(DPD_FF, self).__init__(hoomd_forces)

    def _create_forcefield(self):
        forces = []
        # Bonds
        bond = hoomd.md.bond.Harmonic()
        bond.params["T-T"] = dict(k=self.bond_k, r0=self.bond_r0)
        bond.params["A-X"] = dict(k=0, r0=0)
        forces.append(bond)
        # Angles
        if all([self.angle_k, self.angle_theta0]):
            angle = hoomd.md.angle.Harmonic()
            angle.params["X-A-X"] = dict(k=self.angle_k, t0=self.angle_theta0)
            angle.params["A-X-A"] = dict(k=0, t0=0)
            forces.append(angle)
        # DPD Pairs
        nlist = hoomd.md.nlist.Cell(buffer=0.40, exclusions=["body"])
        dpd = hoomd.md.pair.DPD(nlist=nlist,kT=self.kT,default_r_cut=self.r_cut)
        dpd.params[("X", "X")] = dict(A=self.A, gamma=self.gamma)
        # Add zero pairs
        for pair in [
            ("R", "R"),
            ("T", "T"),
            ("T", "R"),
            ("A", "A"),
            ("A", "X"),
            ("A", "T"),
            ("A", "R"),
            ("X", "R"),
            ("X", "T"),
        ]:
            dpd.params[pair] = dict(A=0,gamma=0.1)
            dpd.params[pair].r_cut = 0.0
        forces.append(dpd)
        return forces

#write new system class to skip pack and shrink step
class RandomSystem(System):
    def __init__(self, molecules, base_units=dict(), buffer=1.00):
        self.buffer = buffer
        super(RandomSystem, self).__init__(
            molecules=molecules, base_units=base_units
        )
    def _build_system(self):
        chain = self.all_molecules
        L = 5.85 #need to figure how to get this from molecules
        box = mb.Box(lengths=np.array([L] * 3) * self.buffer)
        comp = mb.Compound()
        comp.add(chain)
        comp.box = box
        return comp

In [ ]:
# Initialize and write out an ellipsoid chain configuration without relying on Pack()
ellipsoid_chain = EllipsoidChainRand(lengths=10,num_mols=10,lpar=1.0,bead_mass=1.0)
ff = DPD_FF(epsilon=1.0,lpar=1.0,lperp=0.5,A=100,gamma=1.0,kT=2.0,r_cut=1.94,bond_k=100,bond_r0=1.0,angle_k=30,angle_theta0=1.9)
ff.hoomd_forces
system = RandomSystem(molecules=ellipsoid_chain)
system.to_gsd('test.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=1.0,lperp=0.5
)
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

ellipsoid_sim.save_restart_gsd()
ellipsoid_sim.run_NVT(n_steps=1e4, kT=2.0, tau_kt=10*ellipsoid_sim.dt)
ellipsoid_sim.flush_writers()


In [ ]:
# Old initialization using Pack() which calls Packmol. 

ellipsoid_chain = EllipsoidChainRand(lengths=10,num_mols=2,lpar=1.0,bead_mass=1.0)
ff = DPD_FF(epsilon=1.0,lpar=1.0,lperp=0.5,A=100,gamma=1.0,kT=2.0,r_cut=5.0,bond_k=100,bond_r0=1.0,angle_k=30,angle_theta0=1.9)
ff.hoomd_forces
system = Pack(molecules=ellipsoid_chain, density=0.01*u.Unit("nm**-3"),packing_expand_factor=1,edge=0,overlap=0,use_pbc=False,fix_orientation=True)
system.to_gsd('system.gsd')
gsd_path=('ellipsoid-test.gsd')
rigid_frame, rigid = create_rigid_ellipsoid_chain(
    system.hoomd_snapshot,lpar=1.0,lperp=0.5
)
ellipsoid_sim = Simulation(
    initial_state=rigid_frame,
    forcefield=ff.hoomd_forces,
    constraint=rigid,
    dt=0.0003,
    gsd_write_freq=int(100),
    gsd_file_name='trajectory.gsd',
    log_write_freq=int(100),
    log_file_name='log.txt')

ellipsoid_sim.save_restart_gsd()
ellipsoid_sim.run_NVT(n_steps=1e4, kT=2.0, tau_kt=10*ellipsoid_sim.dt)
ellipsoid_sim.flush_writers()

#ff = EllipsoidForcefield(epsilon=1.0,lpar=1.0,lperp=0.5,r_cut=5.0,bond_k=100,bond_r0=0.1,angle_k=30,angle_theta0=1.9)
